# 课程前言

此为 <<人工智能安全>> 课程第一部分: 文本对抗攻击实验部分.

介绍bert，用来进行文本对抗攻击(victim model)


实验准备

In [1]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt

# 进度条显示
from tqdm.notebook import tqdm

import torch
import torch.nn as nn

import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import gensim.downloader as api
from nltk.tokenize import word_tokenize



In [5]:
model_name = "textattack/bert-base-uncased-imdb"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.eval()

# 定义原始文本
text = "The movie was fantastic! The acting was superb and the plot kept me engaged throughout."

# 字符串替换规则
perturbation_map = {
    'a': ['@', 'ä', 'à', 'á'],
    'e': ['3', 'é', 'è'],
    'i': ['1', '!', 'í'],
    'o': ['0', 'ö', 'ó'],
    's': ['$', '5'],
    't': ['7', '+']
}

# 4. 生成对抗样本的函数
def char_perturbation(text, prob=0.1):
    perturbed = []
    for char in text.lower():
        if char in perturbation_map and torch.rand(1).item() < prob:
            choices = perturbation_map[char]
            index = torch.multinomial(torch.ones(len(choices)), 1).item()
            perturbed.append(choices[index])
        else:
            perturbed.append(char)
    return ''.join(perturbed)

def fgsm_perturbation(text, epsilon=0.1):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # 创建一个可求梯度的叶子张量
    embeddings = model.get_input_embeddings()(inputs['input_ids'])
    embeddings = embeddings.detach().clone()
    embeddings.requires_grad = True

    with torch.enable_grad():
        # 使用嵌入表示作为输入
        outputs = model(inputs_embeds=embeddings, attention_mask=inputs['attention_mask'])
        loss = nn.CrossEntropyLoss()(outputs.logits, torch.tensor([1]).to(model.device))

        gradients = torch.autograd.grad(loss, embeddings)[0]
        sign_gradients = gradients.sign()

        # 对嵌入表示进行扰动
        perturbed_embeddings = embeddings + epsilon * sign_gradients

    # 将扰动后的嵌入表示转换回输入 ID
    vocab_size = model.config.vocab_size
    perturbed_input_ids = torch.argmax(torch.matmul(perturbed_embeddings, model.get_input_embeddings().weight.t()), dim=-1)

    perturbed_text = tokenizer.decode(perturbed_input_ids.squeeze(), skip_special_tokens=True)
    return perturbed_text


# 加载预训练的 Word2Vec 模型
# vec_model = api.load("word2vec-google-news-300")

def word_embedding_perturbation(text, num_words=5, prob=0.5):
    tokens = word_tokenize(text.lower())
    perturbed_tokens = []
    for token in tokens:
        if token in vec_model and torch.rand(1).item() < prob:
            similar_words = vec_model.most_similar(token, topn=num_words)
            new_word = similar_words[torch.randint(0, num_words, (1,)).item()][0]
            perturbed_tokens.append(new_word)
        else:
            perturbed_tokens.append(token)
    return " ".join(perturbed_tokens)

# 5. 生成对抗样本
perturbed_text = char_perturbation(text)
# perturbed_text = fgsm_perturbation(text)
# perturbed_text = word_embedding_perturbation(text)
print("Original Text:", text)
print("Perturbed Text:", perturbed_text)

# 6. 测试原始样本和对抗样本的分类结果
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return torch.softmax(outputs.logits, dim=1).detach().numpy()

# 获取预测结果
original_prob = predict(text)
perturbed_prob = predict(perturbed_text)

print("\nOriginal Prediction (pos/neg):", original_prob[0])
print("Perturbed Prediction (pos/neg):", perturbed_prob[0])

Original Text: The movie was fantastic! The acting was superb and the plot kept me engaged throughout.
Perturbed Text: the movie wa$ fantast!c! the acting wa$ superb and thè plot kep+ me èngaged throughout.

Original Prediction (pos/neg): [0.00285912 0.9971409 ]
Perturbed Prediction (pos/neg): [0.48394594 0.5160541 ]


# TextAttack 文本对抗攻击实验指南

在过去的几年里，人们对测试自然语言处理（NLP）模型的对抗鲁棒性越来越感兴趣。该领域的研究涵盖了生成对抗性示例并防御它们的新技术。直接比较这些攻击具有挑战性，因为它们是根据不同的数据和受害者模型进行评估的。

复制早期的工作作为基线需要时间，并且由于缺少源代码而增加了出错的风险。由于出版物中遗漏了一些微小的细节，完美地复制结果也具有挑战性。这些问题给该领域的基准比较带来了挑战。

像 TextAttack 这样的框架已经被开发出来来应对这些挑战。它是一个用于对抗性攻击、数据增强和对抗性训练的 NLP Python 框架。该框架解决了当前的挑战并激发了对抗稳健性方面的进步。这篇文章探讨了 TextAttack，详细介绍了其组件。此外，还将通过深入的代码示例来检查实际实现。

TextAttack 是一个用于自然语言处理（NLP）领域的开源框架，它支持对抗性攻击、数据增强和模型训练。该框架提供了一种标准化的方法来构建和执行针对NLP模型的攻击，同时也可用于评估模型的鲁棒性和进行数据增强以改进模型性能。TextAttack 基于Python 3.6+，并支持GPU加速。

# 0.环境准备

In [1]:
# 安装依赖(建议使用GPU实验)
try:
    import textattack
except ModuleNotFoundError:
    %pip install "textattack[tensorflow]"
    import textattack


import numpy as np 
import torch
import datasets
import transformers

# 设置随机种子, 保证结果可复现
np.random.seed(42)
torch.manual_seed(42)

## 0.1 攻击流程

# 1. 定义目标模型

我们设计的攻击方法要应用在某个具体的模型上，这个被用来测试我们攻击方法的模型就叫做目标模型(受害模型).

目标模型是指在具体的数据集上训练出来的某个模型，因此，我们需要加载数据集和预训练模型来定义目标模型。

In [2]:
from textattack.transformations import WordSwap


class BananaWordSwap(WordSwap):
    """Transforms an input by replacing any word with 'banana'."""

    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        """Returns 'banana', no matter what 'word' was originally.

        Returns a list with one item, since `_get_replacement_words` is intended to
            return a list of candidate replacement words.
        """
        return ["banana"]

In [5]:
# Import the model
import transformers
from textattack.models.wrappers import HuggingFaceModelWrapper

model = transformers.AutoModelForSequenceClassification.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    "textattack/bert-base-uncased-ag-news"
)

model_wrapper = HuggingFaceModelWrapper(model, tokenizer)
# Create the goal function using the model
from textattack.goal_functions import UntargetedClassification

goal_function = UntargetedClassification(model_wrapper)
# Import the dataset
from textattack.datasets import HuggingFaceDataset

dataset = HuggingFaceDataset("ag_news", None, "test")

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like textattack/bert-base-uncased-ag-news is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [11]:
from textattack.datasets import HuggingFaceDataset
from textattack.models.wrappers import HuggingFaceModelWrapper

# 加载数据集, 以二分类情感分类数据集为例
dataset = HuggingFaceDataset("imdb", split="test[:10%]") # 取10%作为示例

# 使用预训练模型
model = HuggingFaceModelWrapper(
    model=AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb"),
    tokenizer=AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
)

ConnectionError: Couldn't reach 'imdb' on the Hub (LocalEntryNotFoundError)

# 2. 定义攻击策略
TextAttack 对抗攻击框架采用模块化架构设计，其核心将对抗样本生成过程解耦为四个正交组件：

1. **目标函数（Goal Function）**

    目标函数通过可量化的指标定义攻击成功的判定标准。以多分类场景的情感分析任务为例，当原始输入为积极情感文本时，目标函数可设定为将模型预测类别成功误导至指定目标类别（如"愤怒"）或任意错误类别。
2. **约束条件（Constraints）**

    约束条件构建对抗扰动的可行性空间，通过多维度规则确保生成样本的有效性。典型约束包括：
    + 词汇级约束：限定修改词数上限（如扰动率≤20%）
    + 语义约束：通过词嵌入相似度或语言模型评分保证语义一致性
    + 语法约束：依赖句法分析维持语法正确性
3. **变换规则（Transformation）**

    变换规则定义文本空间的基本扰动操作，常见策略包括：
    + 词汇替换：基于同义词词库或词嵌入的近邻替换
    + 句法变换：被动语态转换、成分插入/删除等结构修改
    + 字符级攻击：拼写错误注入、不可见字符插入等
4. **搜索方法（Search Method）**

    搜索方法负责在组合爆炸的扰动空间中高效定位有效对抗样本，主要范式包含：
    + 启发式搜索：贪婪搜索、束搜索等
    + 随机搜索：蒙特卡洛采样、遗传算法
    + 梯度引导：基于替代模型梯度信息的优化算法

这样，每一次攻击都会尝试使用 `变换规则` 对输入的文本进行扰动，使其通过 `目标函数`（即判断攻击是否成功），且扰动符合 `约束条件`，最后使用 `搜索方法` 在所有可行的变换结果中，挑出优质的对抗样本。

这种模块化的设计可以使各种对抗攻击策略独立地嵌入到一个系统中，使得我们方便把文献中的方法集成在一起，同时复用各个攻击策略之间相同的部分。

TextAttack不依赖与具体模型，只要模型可以读取字符串并根据目标函数返回一个字符串结果，就可以使用TextAttack进行攻击测试。

In [ ]:
from textattack.attack_recipes import TextFoolerJin2019

attack = TextFoolerJin2019.build(model)

# 可视化攻击参数
print("攻击约束条件: ")
for constraints in attack.constraints:
    print(f"- {constraints}")

print("\n转换方法:")
for transformation in attack.transformation:
    print(f"- {transformation}")

# Goal Funtion 以 AttackedText 对象作为输入，为输入对象打分(类似于损失函数值)，并且判别这次攻击是否满足目标函数定义的成功条件，返回 GoalFuntionResult 对象

# Constraints 以 AttackedText 对象作为输入，返回变换后的 AttackedText 列表，对于每个变换，返回一个 bool 值表示变换是否满足条件

# Transformation 以 Attacked 对象作为输入，返回对于 AttackedText 所有可行变换的列表

# SearchMethod 以初始 GoalFuntionResult 作为输入，返回最终的 GoalFunctionResult

# 3. 执行攻击实验

In [ ]:
# 配置攻击参数
attack_args = textattack.AttackArgs(
    num_examples=20, # 20个攻击样本
    disable_stdout=True
)
attacker = textattack.Attacker(attack, dataset, attack_args)
attack_results = attacker.attack_dataset()

# 4. 实验结果可视化

In [ ]:
success_rate = len(attack_results) / len(dataset)
print(f"攻击成功率: {success_rate: .2%}")

# 扰动程度分析
from textattack.metrics.quality_metrics import Perplexity
perturbation_metrics = Perplexity().calculate(attack_results)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,2, figsize=(12,4))

# 成功率饼图
ax[0].pie([success_rate, 1-success_rate], labels=['成功','失败'], autopct='%1.1f%%')

# 扰动程度直方图
ax[1].hist([r.perturbed_text_perplexity for r in attack_results], bins=15)
ax[1].set_xlabel('Perplexity变化量')

plt.tight_layout()

# 5. 拓展实验

In [ ]:
# 对比攻击方法
methods = {
    "BAE": BERTAttackLi2020,
    "DeepWordBug": DeepWordBugGao2018
}

results = {}
for name, recipe in methods.items():
    attack = recipe.build(model)
    # 执行攻击并记录结果...

介绍结束。还有更多关于 text attack 框架的内容可参考 xxx